## インポート

In [195]:

import sys
import re
import difflib
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             r2_score)
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

from config import (NUMBERS,
                                   NUMBERS3,
                                   NUMBERS4,
                                   NUMBERS4_RESULT,
                                   NUMBERS4_VECTORS,
                                   NUMBERS4_PATTERNS)
import utils


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### NUMBERS4 で予測する

In [4]:
X = []
Y = []
key_0 = 0
key_1 = 0
key_2 = 0
key_3 = 0
key_4 = 0
key_5 = 0

key_6  = 0 #  赤口
key_7  = 0 #  仏滅
key_8  = 0 #  友引
key_9  = 0 #  先勝
key_10 = 0 #  先負
key_11 = 0 #  大安

for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][3]
    _x = NUMBERS4[_i][_time]
    _z = NUMBERS[_i][4]
    for _k, _v in enumerate(_x):
        # if len(list(set(_y) & set(_v))) >= 3:
        if len(list(set(_y) & set(_v))) >= 3 and _z == "先負":
            if _k == 0:
                key_0 += 1
            elif _k == 1:
                key_1 += 1
            elif _k == 2:
                key_2 += 1
            elif _k == 3:
                key_3 += 1
            elif _k == 4:
                key_4 += 1
            elif _k == 5:
                key_5 += 1
                
            if _z == "赤口":
                key_6 += 1
            elif _z == "仏滅":
                key_7 += 1
            elif _z == "友引":
                key_8 += 1
            elif _z == "先勝":
                key_9 += 1
            elif _z == "先負":
                key_10 += 1
            elif _z == "大安":
                key_11 += 1
            
            
            _format = "{0}回 {1}:{2}({3}) [{4}]".format(_time, _y, _x[_k], _k, _z)
            # print(_format)
    # Y.append(_y)
    # X.append(_x)
    pass
_format1 = "前回:{0}個, 前々回:{1}個, 前回曜日:{2}個, 前々回曜日:{3}個, 前回六曜:{4}個, 前々回六曜:{5}個".format(key_0, key_1, key_2, key_3, key_4, key_5)
_format2 = "赤口:{0}個, 仏滅:{1}個, 友引:{2}個 先勝:{3}個, 先負:{4}個, 大安:{5}個".format(key_6, key_7, key_8, key_9, key_10, key_11)
_sum = sum([key_0, key_1, key_2, key_3, key_4, key_5])

print(_format1)

前回:9個, 前々回:7個, 前回曜日:8個, 前々回曜日:5個, 前回六曜:8個, 前々回六曜:14個


#### 6. 抽選数を特徴量とする

In [2]:
X = []
Y = []
TIME = []
for _i in range(len(NUMBERS4_RESULT)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][3]
    _x = NUMBERS4_RESULT[_i][_time]
    _z = NUMBERS[_i][4]
    if _z == "先勝":
        Y.append(_y)
        X.append(_x)
        TIME.append(_time)

#### 設定

In [388]:
OLD_WEEK = "先勝"
TYPE            = "-"

In [402]:
PATTERN = "pattern{0}".format(13)

##### 特徴量

In [403]:
VEC = utils.get_vectors(OLD_WEEK, TYPE, NUMBERS4_VECTORS)
NUM  = utils.get_patterns(NUMBERS4_PATTERNS, PATTERN, TYPE)
print(NUM)

[3, 3]


##### セット

In [391]:
X       = []
Y       = []
TIME = []
k1 = VEC[0]
k2 = VEC[1]
k3 = VEC[2]
n1 = NUM[0]
n2 = NUM[1]

##### 計算

In [392]:
X = []
Y = []
TIME = []
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][3]
    _x1 = NUMBERS4[_i][_time][k1]
    _x2 = NUMBERS4[_i][_time][k2]
    _x３ = NUMBERS4[_i][_time][k3]
    _z = NUMBERS[_i][4]
    if not len(list(set(_y) & set(_x1))) >= n1 and not len(list(set(_y) & set(_x2))) >= n2  and _z == OLD_WEEK:
        Y.append(_y)
        X.append([_x1, _x2, _x3])
        TIME.append(_time)

## 予測

In [370]:
y = pd.DataFrame(Y)
x = pd.DataFrame(X)

In [371]:
len(y)

28

In [372]:
len(TIME)

28

In [373]:
model = linear_model.LinearRegression()
model.fit(X, Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [374]:
px = x
py = model.predict(px)

In [375]:
for _i in range(len(py)):
    _py = str(int(py[_i]))
    for _k, _v in enumerate(NUMBERS):
        if _v[0] == TIME[_i]:
            if len(list(set(_v[3]) & set(_py))) >= 3:
                print(TIME[_i], _v[3], py[_i])
            # print(TIME[_i], _v[3], py[_i])

## 決定係数

In [376]:
r2_score(y, py)

0.1147671125693881

In [377]:
model.score(x, y)

0.1147671125693881

In [378]:
model.coef_[0]

-0.1580460720879233

## 予測

In [379]:
pkl_file = "data/numbers4_20180710.pkl"

In [380]:
data = [float("7278"), float("6830"), float("2154")]

#### 抽選数

In [19]:
from statistics import mean, median,variance,stdev

X0 = []
X1 = []
X2 = []
X3 = []
for _i in range(len(NUMBERS4_RESULT)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][3]
    _x0 = NUMBERS4_RESULT[_i][_time][0]
    _x1 = NUMBERS4_RESULT[_i][_time][1]
    _x2 = NUMBERS4_RESULT[_i][_time][2]
    _x3 = NUMBERS4_RESULT[_i][_time][3]
    X0.append(int(_x0))
    X1.append(int(_x1))
    X2.append(int(_x2))
    X3.append(int(_x3))
    

x0 = int(mean(X0))
x1 = int(mean(X1))
x2 = int(mean(X2))
x3 = int(mean(X3))
print(x0, x1, x2, x3)

# 20180618
# data = [float("1"), float("1"), float("1"), float("1")]
data = [float(str(x0)), float(str(x1)), float(str(x2)), float(str(x3))]

22 270 71 1187


In [381]:
joblib.dump(model, pkl_file)

['data/numbers4_20180710.pkl']

In [382]:
clf = joblib.load(pkl_file)

In [383]:
processor = clf.predict([data])

In [384]:
processor

array([4003.6458849])